In [250]:

# import dependencies
import IPython
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline
import dlib
import numpy as np
import imutils
from scipy.spatial import distance as dist
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from scipy.spatial.distance import euclidean as dist
from IPython.display import Javascript, display
from scipy.spatial import distance as dist
from imutils import face_utils
from imutils.face_utils import FaceAligner
from scipy.spatial import distance as dist
from imutils import face_utils
import base64
from PIL import Image
import warnings 

warnings.filterwarnings("ignore")



In [251]:
# Play an audio beep. Any audio URL will do.
alert='new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()'
bienvenue='new Audio("https://upload.wikimedia.org/wikipedia/commons/1/1a/Bienvenue-chez-votre-applicati1651880044.ogg").play()'
pause='new Audio("https://upload.wikimedia.org/wikipedia/commons/7/7e/Penser-a-prendre-une-petite-pa1651880436.ogg").play()'
reveil='new Audio("https://upload.wikimedia.org/wikipedia/commons/2/23/R%C3%A9veillez-vousr%C3%A9veillez-vous-1651880280_%281%29.ogg").play()'


In [252]:
model=load_model(r"C:\Users\User\Documents\Desktop\Projet\somnolance.keras")
figure = plt.figure(figsize=(5, 5))
face_cascade = cv2.CascadeClassifier(r'C:\Users\User\Documents\Desktop\Projet\Hardcascad_file\haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(r'C:\Users\User\Documents\Desktop\Projet\Hardcascad_file\haarcascade_eye.xml')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r'C:\Users\User\Documents\Desktop\Projet\shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(predictor, desiredFaceWidth=256)

In [253]:


def calculate_lip(lips):
    # Vérifier que les points clés sont valides
    if len(lips) < 7:
        raise ValueError("Le tableau 'lips' doit contenir au moins 7 points.")

    # Calculer les distances
    dist1 = dist(lips[2], lips[6])  # Hauteur de l'ouverture de la bouche
    dist2 = dist(lips[0], lips[4])  # Largeur de la bouche

    # Éviter une division par zéro
    if dist2 == 0:
        return 0.0

    # Calculer le ratio LAR
    LAR = float(dist1 / dist2)

    return LAR

In [254]:

def js_to_image(js_reply):
    """
    Convertit une image JavaScript (base64) en une image OpenCV au format BGR.

    Args:
        js_reply (str): Chaîne JavaScript contenant l'image au format base64.

    Returns:
        img (numpy.ndarray): Image OpenCV au format BGR.

    Raises:
        ValueError: Si le format de `js_reply` est invalide.
    """
    try:
        # Extraire la partie base64 de la chaîne JavaScript
        if ',' not in js_reply:
            raise ValueError("Format de js_reply invalide. Attendu: 'data:image/...;base64,...'")
        
        # Décoder l'image en base64
        image_bytes = base64.b64decode(js_reply.split(',')[1])
        
        # Convertir les octets en tableau numpy
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        
        # Décoder le tableau numpy en image OpenCV au format BGR
        img = cv2.imdecode(jpg_as_np, flags=cv2.IMREAD_COLOR)
        
        if img is None:
            raise ValueError("Échec de la décodage de l'image.")
        
        return img
    except Exception as e:
        raise ValueError(f"Erreur lors de la conversion de l'image : {e}")


def bbox_to_bytes(bbox_array):
    """
    Convertit un tableau numpy (boîte englobante) en une chaîne base64 pour l'affichage.

    Args:
        bbox_array (numpy.ndarray): Tableau numpy (pixels) contenant la boîte englobante au format RGBA.

    Returns:
        str: Chaîne base64 de l'image PNG encodée.

    Raises:
        ValueError: Si `bbox_array` n'est pas un tableau numpy valide.
    """
    try:
        # Vérifier que l'entrée est un tableau numpy
        if not isinstance(bbox_array, np.ndarray):
            raise ValueError("bbox_array doit être un tableau numpy.")
        
        # Convertir le tableau en image PIL
        bbox_PIL = Image.fromarray(bbox_array, 'RGBA')
        
        # Utiliser un tampon mémoire pour enregistrer l'image
        with io.BytesIO() as iobuf:
            # Enregistrer l'image au format PNG dans le tampon
            bbox_PIL.save(iobuf, format='PNG')
            # Encoder l'image en base64
            bbox_bytes = base64.b64encode(iobuf.getvalue()).decode('utf-8')
        
        # Formater la chaîne de retour
        return f'data:image/png;base64,{bbox_bytes}'
    except Exception as e:
        raise ValueError(f"Erreur lors de la conversion de la boîte englobante : {e}")

In [255]:
def video_stream():
    js = Javascript(
'''
var video;
var div = null;
var stream;
var captureCanvas;
var imgElement;
var labelElement;

var pendingResolve = null;
var shutdown = false;

/**
 * Nettoie et supprime les éléments DOM créés pour le flux vidéo.
 */
function removeDom() {
    if (stream) {
        stream.getVideoTracks()[0].stop(); // Arrête le flux vidéo
    }
    if (video) video.remove();
    if (div) div.remove();
    if (imgElement) imgElement.remove();
    if (captureCanvas) captureCanvas.remove();
    if (labelElement) labelElement.remove();

    video = null;
    div = null;
    stream = null;
    imgElement = null;
    captureCanvas = null;
    labelElement = null;
}

/**
 * Fonction appelée à chaque frame d'animation pour capturer une image.
 */
function onAnimationFrame() {
    if (shutdown) return;

    window.requestAnimationFrame(onAnimationFrame);

    if (pendingResolve) {
        let result = "";
        if (!shutdown && captureCanvas && video) {
            const context = captureCanvas.getContext('2d');
            context.drawImage(video, 0, 0, 640, 480); // Capture une image de la vidéo
            result = captureCanvas.toDataURL('image/jpeg', 0.8); // Convertit en base64
        }
        const resolve = pendingResolve;
        pendingResolve = null;
        resolve(result); // Renvoie l'image capturée
    }
}

/**
 * Crée les éléments DOM nécessaires pour afficher le flux vidéo.
 */
async function createDom() {
    if (div !== null) return stream;

    // Crée un conteneur pour la vidéo
    div = document.createElement('div');
    div.style.border = '2px solid black';
    div.style.padding = '3px';
    div.style.width = '100%';
    div.style.maxWidth = '600px';
    document.body.appendChild(div);

    // Crée un élément pour afficher le statut
    const modelOut = document.createElement('div');
    modelOut.innerHTML = "<span>Status:</span>";
    labelElement = document.createElement('span');
    labelElement.innerText = 'No data';
    labelElement.style.fontWeight = 'bold';
    modelOut.appendChild(labelElement);
    div.appendChild(modelOut);

    // Crée l'élément vidéo
    video = document.createElement('video');
    video.style.display = 'block';
    video.width = div.clientWidth - 6;
    video.setAttribute('playsinline', '');
    video.onclick = () => { shutdown = true; }; // Permet de cliquer pour arrêter

    // Demande l'accès à la webcam
    try {
        stream = await navigator.mediaDevices.getUserMedia({ video: { facingMode: "environment" } });
        div.appendChild(video);

        // Crée un élément pour afficher les images superposées
        imgElement = document.createElement('img');
        imgElement.style.position = 'absolute';
        imgElement.style.zIndex = 1;
        imgElement.onclick = () => { shutdown = true; };
        div.appendChild(imgElement);

        // Ajoute des instructions pour l'utilisateur
        const instruction = document.createElement('div');
        instruction.innerHTML = 
            '<span style="color: red; font-weight: bold;">' +
            'When finished, click here or on the video to stop this demo</span>';
        div.appendChild(instruction);
        instruction.onclick = () => { shutdown = true; };

        video.srcObject = stream;
        await video.play();

        // Crée un canvas pour capturer les images
        captureCanvas = document.createElement('canvas');
        captureCanvas.width = 640; // Largeur fixe
        captureCanvas.height = 480; // Hauteur fixe
        window.requestAnimationFrame(onAnimationFrame);

        return stream;
    } catch (error) {
        console.error("Erreur lors de l'accès à la webcam :", error);
        removeDom();
        throw error;
    }
}

/**
 * Capture une image du flux vidéo et renvoie les données.
 * @param {string} label - Le texte à afficher comme statut.
 * @param {string} imgData - Les données de l'image à superposer.
 * @returns {Promise<Object>} - Les données de l'image capturée et les timings.
 */
async function stream_frame(label, imgData) {
    if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
    }

    const preCreate = Date.now();
    await createDom();

    const preShow = Date.now();
    if (label) {
        labelElement.innerHTML = label;
    }

    if (imgData) {
        const videoRect = video.getClientRects()[0];
        imgElement.style.top = `${videoRect.top}px`;
        imgElement.style.left = `${videoRect.left}px`;
        imgElement.style.width = `${videoRect.width}px`;
        imgElement.style.height = `${videoRect.height}px`;
        imgElement.src = imgData;
    }

    const preCapture = Date.now();
    const img = await new Promise((resolve) => {
        pendingResolve = resolve;
    });
    shutdown = false;

    return {
        create: preShow - preCreate,
        show: preCapture - preShow,
        capture: Date.now() - preCapture,
        img: img
    };
}  

''') 
    display(js)
  
def video_frame(label, bbox):
    data = eval_js(f'stream_frame("{label}", "{bbox}")')
    return data

In [256]:
def video_frame(label, bbox):
    """
    Capture une image à partir du flux vidéo de la webcam en utilisant JavaScript.

    Args:
        label (str): Le texte à afficher comme statut dans l'interface utilisateur.
        bbox (str): Les données de l'image à superposer (par exemple, une boîte englobante).

    Returns:
        dict: Un dictionnaire contenant les données de l'image capturée et les timings.
              Format : {'create': temps, 'show': temps, 'capture': temps, 'img': base64}.

    Raises:
        ValueError: Si `label` ou `bbox` ne sont pas des chaînes de caractères valides.
        RuntimeError: Si une erreur survient lors de l'exécution du code JavaScript.
    """
    # Validation des entrées
    if not isinstance(label, str) or not isinstance(bbox, str):
        raise ValueError("Les arguments 'label' et 'bbox' doivent être des chaînes de caractères.")

    try:
        # Exécuter le code JavaScript pour capturer une image
        data = eval_js(f'stream_frame("{label}", "{bbox}")')
        
        # Vérifier que les données retournées sont valides
        if not data or not isinstance(data, dict):
            raise RuntimeError("Aucune donnée valide retournée par le code JavaScript.")
        
        return data
    except Exception as e:
        # Capturer et relancer les erreurs avec un message clair
        raise RuntimeError(f"Erreur lors de la capture de l'image : {e}")

In [257]:
import webview

def video_frame(label, bbox):
    """
    Capture une image à partir du flux vidéo de la webcam en utilisant JavaScript.

    Args:
        label (str): Le texte à afficher comme statut dans l'interface utilisateur.
        bbox (str): Les données de l'image à superposer (par exemple, une boîte englobante).

    Returns:
        dict: Un dictionnaire contenant les données de l'image capturée et les timings.
              Format : {'create': temps, 'show': temps, 'capture': temps, 'img': base64}.

    Raises:
        ValueError: Si `label` ou `bbox` ne sont pas des chaînes de caractères valides.
        RuntimeError: Si une erreur survient lors de l'exécution du code JavaScript.
    """
    # Validation des entrées
    if not isinstance(label, str) or not isinstance(bbox, str):
        raise ValueError("Les arguments 'label' et 'bbox' doivent être des chaînes de caractères.")

    try:
        # Créer une fenêtre Web pour exécuter le JavaScript
        window = webview.create_window("Webcam Stream", html="<h1>Loading...</h1>")
        webview.start()

        # Injecter le code JavaScript
        js_code = f'''
        stream_frame("{label}", "{bbox}")
            .then(data => {{
                const output = JSON.stringify(data);
                console.log(output);  // Afficher les données dans la console
                window.pywebview.api.returnData(output);
            }});
        '''
        window.evaluate_js(js_code)

        # Attendre que le résultat soit disponible
        if 'video_frame_result' in globals():
            data = globals()['video_frame_result']
            return data
        else:
            raise RuntimeError("Aucune donnée retournée par le code JavaScript.")
    except Exception as e:
        # Capturer et relancer les erreurs avec un message clair
        raise RuntimeError(f"Erreur lors de la capture de l'image : {e}")

In [258]:
import dlib
import cv2
import numpy as np
from scipy.spatial import distance as dist
from imutils import face_utils

# Initialisation des détecteurs et prédicteurs
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"shape_predictor_68_face_landmarks.dat")
fa = face_utils.FaceAligner(predictor, desiredFaceWidth=256)

# Seuils pour la détection de bâillement et de somnolence
LIP_AR_THRESH = 0.4  # Seuil pour le ratio d'ouverture de la bouche
EYE_AR_THRESH = 0.25  # Seuil pour le ratio d'ouverture des yeux
LIP_PER_FRAME = 1  # Nombre de frames consécutifs pour déclencher une alerte

# Fonction pour calculer le ratio d'ouverture de la bouche
def calculate_lip(lip_points):
    dist1 = dist.euclidean(lip_points[2], lip_points[6])
    dist2 = dist.euclidean(lip_points[0], lip_points[4])
    return dist1 / dist2

# Démarrer le flux vidéo
def video_stream():
    global video_capture
    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        raise ValueError("La caméra ne peut pas être ouverte.")

def video_frame():
    # Lire une image du flux vidéo
    ret, frame = video_capture.read()
    if not ret:
        return False
    return frame

# Définir une fonction pour afficher l'alerte
def alert():
    print("ALERTE: Somnolence détectée!")

def reveil():
    print("Réveil: Le conducteur est alerte.")

def pause():
    print("Pause: Le conducteur doit se reposer.")

def bbox_to_bytes(image):
    # Convertir l'image en PNG et l'encoder en base64
    _, buffer = cv2.imencode('.png', image)
    return buffer.tobytes()

# Initialisation des variables
counter = 0
bbox = ''

# Démarrer le flux vidéo
video_stream()

# Boucle principale de détection
while True:
    # Capturer une image du flux vidéo
    img = video_frame()
    if img is False:
        break

    # Créer un overlay transparent pour la boîte englobante
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Vérification si l'image est bien en niveaux de gris (8 bits)
    if len(gray.shape) != 2:
        print("Erreur: l'image n'est pas en niveaux de gris")
        continue  # Ignorer cette image et passer à la suivante

    # Détecter les visages
    rects = detector(gray, 2)
    for rect in rects:
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        face_aligned = fa.align(img, gray, rect)
        face_aligned_gray = cv2.cvtColor(face_aligned, cv2.COLOR_BGR2GRAY)

        # Détecter les yeux
        eyes = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml").detectMultiScale(face_aligned_gray, 1.1, 4)
        predictions = []
        for (ex, ey, ew, eh) in eyes:
            eye = face_aligned[ey:ey+eh, ex:ex+ew]
            eye = cv2.resize(eye, (32, 32))
            eye = np.expand_dims(eye, axis=0)
            
            # Ici, vous devez charger ou définir un modèle pour la prédiction des yeux
            # Exemple : model = load_model('eye_model.h5') avant d'utiliser model.predict()
            # ypred = model.predict(eye)
            ypred = np.array([0])  # Remplacez par votre prédiction du modèle
            ypred = np.argmax(ypred[0], axis=0)
            predictions.append(ypred)

        # Vérifier si les yeux sont fermés
        if all(i == 0 for i in predictions):
            cv2.rectangle(bbox_array, (x, y), (x+w, y+h), (255, 0, 0), 8)
            cv2.putText(bbox_array, 'Sleeping', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3)
            alert()
            reveil()
        else:
            cv2.rectangle(bbox_array, (x, y), (x+w, y+h), (0, 255, 0), 8)
            cv2.putText(bbox_array, 'Not Sleeping', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

        # Détecter les bâillements
        lip_points = [60, 61, 62, 63, 64, 65, 66, 67]
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        lip_shape = shape[lip_points]
        LAR = calculate_lip(lip_shape)

        # Dessiner les contours des lèvres
        lip_hull = cv2.convexHull(lip_shape)
        cv2.drawContours(bbox_array, [lip_hull], -1, (0, 255, 0), 1)

        # Vérifier si la bouche est ouverte
        if LAR > LIP_AR_THRESH:
            counter += 1
            if counter > LIP_PER_FRAME:
                cv2.putText(bbox_array, "YAWNING ALERT!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                pause()
        else:
            counter = 0

        # Afficher le ratio d'ouverture de la bouche
        cv2.putText(bbox_array, "LAR: {:.2f}".format(LAR), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Convertir l'overlay en bytes
    bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)

    # Mettre à jour la boîte englobante pour la prochaine frame
    bbox = bbox_bytes

    # Afficher l'image
    cv2.imshow("Frame", img)

    # Quitter si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la capture et fermer les fenêtres
video_capture.release()
cv2.destroyAllWindows()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.